In [1]:
import glob
import numpy as np
import scipy as sp
import einops as eo
import matplotlib.pyplot as plt
import math
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation
import magpylib as magpy
import os


M0 = 1480 #mT
shape = [3*25.4/16, 25.4/8] #radius and height
x0 = np.array([0,0,10, 0,0,1])

In [2]:
def B_dipole(position, rotation, M0, shape):
    R = np.sqrt(np.sum(position**2, axis=1))
    B = (M0 * (shape[0]) ** 2 * shape[1] / (16)) * (
        (
            3
            * position
            / R[:, np.newaxis] ** 5
            * (eo.einsum(position, rotation, "sensor dim,  dim -> sensor"))[
                :, np.newaxis
            ]
        )
        - rotation[np.newaxis, :] / (R[:, np.newaxis] ** 3)
    )
    return B


def getField_dipole(x, positions, M0, shape):
    position = x[:3]
    axis = x[3:]
    return B_dipole(positions - position, axis, M0, shape)


def getField_dipole_fixed(x, positions, M0, shape):
    position = x[:3]
    axis = x[3:]
    return B_dipole(positions - position, axis, M0, shape)


def cost_dipole(x, B, positions, M0, shape):
    diff = getField_dipole(x, positions, M0, shape) - B
    return np.sum((diff) ** 2)


def getField_cylinder(x, positions, M0, shape):
    B=magpy.getB(
        sources="Cylinder",
        position=x[:3],
        orientation=Rotation.align_vectors(x[3:], np.array([0, 0, 1]))[0],
        observers=positions,
        dimension=shape,
        polarization=(0, 0, M0),
    )
    return B

def cost_cylinder(x, B, positions, M0, shape):
    diff = getField_cylinder(x, positions, M0, shape) - B
    return np.sum((diff) ** 2)

def minimize(x0, B, positions, M0, shape, *args):
    #print("Starting mimimization")
    b_args = (B, positions, M0, shape)
    cons = [{"type": "eq", "fun": lambda x: x[3] ** 2 + x[4] ** 2 + x[5] ** 2 - 1}]
    bounds = [(-100, 100), (-100, 100), (0, 100), (-1, 1), (-1, 1), (-1, 1)]
    res = sp.optimize.minimize(
        fun=cost_dipole, x0=x0, args=b_args, tol=1e-100, constraints=cons, bounds=bounds, *args
    ).x  
    #print(f"Finished mimimization with shape {b_args[3]} at {res}")
    return res

def circle_radius(x1, y1, x2, y2, x3, y3):
    # Calculate lengths of sides of the triangle formed by the three points
    a = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    b = math.sqrt((x3 - x2) ** 2 + (y3 - y2) ** 2)
    c = math.sqrt((x3 - x1) ** 2 + (y3 - y1) ** 2)

    # Calculate the semi-perimeter of the triangle
    s = (a + b + c) / 2

    # Calculate the area of the triangle using Heron's formula
    area = math.sqrt(s * (s - a) * (s - b) * (s - c))

    # Calculate the radius of the circle using the formula: radius = (abc) / (4 * area)
    radius = (a * b * c) / (4 * area)
    
    return radius

def comparison_plot(locact, locpred, axis, relative):

    if axis == 'x':
        ax = 0
    elif axis == 'y':
        ax = 1
    else:
        ax = 2

    index = list(range(len(locact[:,ax])))
    act = locact[:, ax]
    pred = locpred[:, ax]

    rel = ''

    if relative==True:
        act = [x - locact[0, ax] for x in locact[:, ax]]
        pred = [x - locpred[0, ax] for x in locpred[:, ax]]
        rel = ' (Relative)'

    # Plotting
    plt.figure(figsize=(8, 6))
    plt.scatter(index, pred, label='Predicted' + rel)
    plt.scatter(index, act, label='Actual' + rel)
    plt.xlabel('Index')
    plt.ylabel(axis + '-component')
    plt.title('Comparison of '+axis+ ' Location' + rel+ ': Predicted vs Actual')
    plt.legend()
    plt.grid(True)
    plt.show()

    return

def comp_plot3D(locact, locpred, xact, yact, axis, relative):

    if axis == 'x':
        ax = 0
    elif axis == 'y':
        ax = 1
    else:
        ax = 2

    index = list(range(len(locact[:,ax])))
    act = locact[:, ax]
    pred = locpred[:, ax]

    rel = ''

    if relative==True:
        act = [x - locact[0, ax] for x in locact[:, ax]]
        pred = [x - locpred[0, ax] for x in locpred[:, ax]]
        rel = ' (Relative)'

    fig = go.Figure(data=[
        go.Scatter3d(x=xact, y=yact, z=pred, name='Predicted' + rel),
        go.Scatter3d(x=xact, y=yact, z=act, name='Actual' + rel)])
    fig.show()

    return

def displacement_plot(actual, predicted):

    dispx = np.array(predicted[:,0]-predicted[:,0][0])
    dispy = np.array(predicted[:,1]-predicted[:,1][0])
    dispz = np.array(predicted[:,2]-predicted[:,2][0])
    disp = np.sqrt(dispx**2+dispy**2+dispz**2)
    actdispx = np.array(actual[:,0]-actual[:,0][0])
    actdispy = np.array(actual[:,1]-actual[:,1][0])
    actdispz = np.array(actual[:,2]-actual[:,2][0])
    actdisp = np.sqrt(actdispx**2+actdispy**2+actdispz**2)

    plt.figure(figsize=(8, 6))
    plt.scatter(list(range(len(disp))), disp, label='Predicted Displacement')
    plt.scatter(list(range(len(actdisp))), actdisp, label='Actual Displacement')
    plt.xlabel('Index')
    plt.ylabel('Displacement (mm)')
    plt.title('Magnet Displacement')
    plt.legend()
    plt.grid(True)
    plt.show()

    return

def importfitting(folder):

    file_list = sorted(glob.glob(f'{folder}/*.npz'))
    len(file_list)
    data = np.load(file_list[0])
    pos = data['pos'][0]

    locpred = [np.array([0,0,0])]
    anglepred = [np.array([0,0,0])]
    Bstdev = [np.array(pos)]
    for x in range(len(file_list)):
        data = np.load(file_list[x])
        mags = data['mags']
        pos = data['pos'][0]
        B = np.mean(mags, axis=0)
        mags_stdev = np.std(mags, axis=0)
        x_res = minimize(x0, B, pos, M0, shape)
        locpred = np.append(locpred, [x_res[0:3]], axis=0)
        anglepred = np.append(anglepred, [x_res[3:6]], axis=0)
        Bstdev = np.append(Bstdev, [mags_stdev], axis=0)
    locpred = np.delete(locpred, 0, axis=0)
    anglepred = np.delete(anglepred, 0, axis=0)
    Bstdev = np.delete(Bstdev, 0 ,axis=0)

    return locpred, anglepred, Bstdev

def actualangle(xind, yind, xscal, xcon, yscal, ycon, isoff):

    angact = [np.array([0, 0, 0])]
    for y in range(yind):
        for x in range(xind):
            R = Rotation.from_euler('ZYX' ,np.array([0, yscal*y-ycon, xscal*x-xcon]), degrees=True)
            norm = np.array([[0, 0, 1]])
            rotated = R.apply(norm, inverse=False)[0]
            #add = np.array([np.sin(np.deg2rad(2*y-4)), -np.sin(np.deg2rad(2*x-4)), np.cos(np.deg2rad(2*y-4))*np.cos(np.deg2rad(2*x-4))])
            angact = np.append(angact, [rotated], axis=0)
    angact = np.delete(angact, 0, axis=0)
    locact = angact*isoff

    return angact, locact

In [3]:
folder = os.path.join('..', './MLX06242024_D32N52_AboutZ_Logs')
locpred, anglepred, Bstdev = importfitting(folder)
np.append(locpred, anglepred, axis=1)

array([[-1.94772585e+00, -1.62631812e+01,  1.01353128e+01,
        -1.20658647e-01, -6.42304715e-02,  9.90613920e-01],
       [-1.94783351e+00, -1.62754511e+01,  1.01249296e+01,
        -1.20614442e-01, -6.62577851e-02,  9.90485771e-01],
       [-1.95356900e+00, -1.62603842e+01,  1.01350539e+01,
        -1.21163310e-01, -6.34781271e-02,  9.90600817e-01],
       [-1.95183560e+00, -1.62658693e+01,  1.01323305e+01,
        -1.21134033e-01, -6.48390960e-02,  9.90516248e-01],
       [-1.95001940e+00, -1.62596953e+01,  1.01368654e+01,
        -1.20760452e-01, -6.40216062e-02,  9.90615035e-01],
       [-1.95292268e+00, -1.62659319e+01,  1.01311195e+01,
        -1.21048845e-01, -6.46750347e-02,  9.90537388e-01],
       [-1.95296956e+00, -1.62615188e+01,  1.01347915e+01,
        -1.21329267e-01, -6.40216622e-02,  9.90545524e-01],
       [-1.94910880e+00, -1.62632934e+01,  1.01338503e+01,
        -1.20718453e-01, -6.44236895e-02,  9.90594086e-01],
       [-1.95134190e+00, -1.62700140e+01,  1.012

In [4]:
print(len(locpred))

125


In [5]:
Dist = 13.77 #mm
SensorHeight = 0.85
zact = Dist - shape[1]/2 - SensorHeight
radact = np.sqrt(800)/2
point = (radact, 0, zact)

# Function to rotate a 3D point around the z-axis
def rotate_point_z_axis(point, angle_degrees):
    angle_radians = np.radians(angle_degrees)
    rotation_matrix = np.array([
        [np.cos(angle_radians), -np.sin(angle_radians), 0],
        [np.sin(angle_radians), np.cos(angle_radians), 0],
        [0, 0, 1]
    ])
    return np.dot(rotation_matrix, point)

# Generate points rotating every 60 degrees
angles = [60 * i for i in range(6)]
locact = np.array([rotate_point_z_axis(point, angle) for angle in angles])

# Print the points
for i, p in enumerate(locact):
    print(f"Point {i}: {p}")
    
actang = np.array([(0, 0, -1) for _ in range(len(locact))])

Point 0: [14.14213562  0.         11.3325    ]
Point 1: [ 7.07106781 12.24744871 11.3325    ]
Point 2: [-7.07106781 12.24744871 11.3325    ]
Point 3: [-1.41421356e+01  1.73191211e-15  1.13325000e+01]
Point 4: [ -7.07106781 -12.24744871  11.3325    ]
Point 5: [  7.07106781 -12.24744871  11.3325    ]


In [23]:
size = 100

measured = go.Cone(x=locpred[:,0], y=locpred[:,1], z=locpred[:,2], u=anglepred[:,0], v=anglepred[:,1], w=anglepred[:,2], 
                             sizemode="absolute", sizeref=200, colorscale=[(0, 'red'), (1, 'red')])

calculated = go.Cone(x=locact[:,0], y=locact[:,1], z=locact[:,2], u=actang[:,0], v=actang[:,1], w=actang[:,2], 
                             sizemode="absolute", sizeref=0.01, colorscale=[(0, 'blue'), (1, 'blue')])

layout = go.Layout(scene=dict(aspectmode='cube', zaxis=dict(range=[9,13]), xaxis=dict(range=[-20,20]),
                   yaxis=dict(range=[-20,20])))

fig = go.Figure(data=[measured])

fig.show()

In [28]:
a = 0
b = 1

measured2D = go.Scatter(x=locpred[:,a], y=locpred[:,b], mode='markers')
calc2D = go.Scatter(x=locact[:,a], y=locact[:,b], mode='markers')
fig2 = go.Figure(data=[measured2D])
fig2.update_yaxes(scaleanchor='x')
fig2.show()

In [26]:
a = 0
b = 2

measured2D = go.Scatter(x=locpred[:,a], y=locpred[:,b], mode='markers')
calc2D = go.Scatter(x=locact[:,a], y=locact[:,b], mode='markers')
fig2 = go.Figure(data=[measured2D])
fig2.update_yaxes(scaleanchor='x')
fig2.show()

In [31]:
a = 1
b = 2

measured2D = go.Scatter(x=locpred[:,a], y=locpred[:,b], mode='markers')
calc2D = go.Scatter(x=locact[:,a], y=locact[:,b], mode='markers')
fig2 = go.Figure(data=[measured2D])
fig2.update_yaxes(scaleanchor='x')
fig2.show()